## Preprocessing of the Log File
#### We will preprocessing and wrangle the messy log files and cleanly populate the dataframe. This dataframe will be used to visualise our data in our web app using Dash Plotly framework.

### Import libraries

In [55]:
import pandas as pd
import numpy as np
from datetime import datetime

### Read contents of the file to the program

In [ ]:
file = open("gerror.txt", "r")
contents = ""
for i in range(5000):
    contents += str(file.readline())

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/pako/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-56-1cd3dc4ebeaf>", line 1, in <module>
    file = open("gerror.txt", "r")
FileNotFoundError: [Errno 2] No such file or directory: 'gerror.txt'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pako/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pako/.local/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  Fil

### Data wrangling

In [56]:
# split string respect to newline 
lines = contents.split('\n') # list of log entries

date_time_list = []
severity_level_list = []
dirty_list = []

for line in lines:
    pieces = line.split(" ")
    if len(pieces) > 11: # some lines are less than the minimum required length
        date_time_list.append(" ".join([pieces[dt] for dt in range(5)]))
        severity_level_list.append(pieces[5])
        dirty_list.append(" ".join([pieces[e] for e in range(6,len(pieces),1)]))

# Clean the actual error messages
error_msg_list = []
error_msg = ""
for dirty in dirty_list:
    dirty_pieces = dirty.split(" ")
    error_msg = ""
    for dirt_piece in dirty_pieces:
        if not any(char.isdigit() for char in dirt_piece):
            if not any(not char.isalnum() for char in dirt_piece):
                error_msg += dirt_piece + " "
    if error_msg:
        error_msg_list.append(error_msg)

# Formate the date
MONTH_MAP = {
    # map words to digits
    "Jan":"01","Feb":"02","Mar":"03","Apr":"04","May":"05","Jun":"06",
    "Jul":"07","Aug":"08","Sep":"09","Oct":"10","Nov":"11","Dec":"12"
}

day_of_week_list = []
date_list = []
for date_time_str in date_time_list:
    date_time_str = date_time_str.replace('[','')
    date_time_str = date_time_str.replace(']','')
    date_time_pieces = date_time_str.split(" ")
    day_of_week_list.append(date_time_pieces[0])
    formatted_date_str = date_time_pieces[2] +"/"+ MONTH_MAP[date_time_pieces[1][:3]] +"/"+ date_time_pieces[4][2:]
    # convert date string to actual date object and append to date list
    date_list.append(datetime.strptime(formatted_date_str, '%d/%m/%y').date())
          
data = {
    "date_time": date_list,
    "day_of_the_week": day_of_week_list,
    "severity_level": severity_level_list,
    "error_messages": error_msg_list
}

df = pd.DataFrame(data)
df['error_messages'] = df['error_messages'].apply(lambda msg: msg.lower()) 
df


ERROR! Session/line number was not unique in database. History logging moved to new session 35


,date_time,day_of_the_week,error_messages,severity_level
0,2019-05-02,Thu,user authentication failure for password misma...,[auth_basic:error]
1,2019-05-27,Mon,php undefined salt in on line,[:error]
2,2019-05-27,Mon,php undefined salt in on line,[:error]
3,2019-06-07,Fri,php undefined salt in on line,[:error]
4,2019-06-07,Fri,php undefined salt in on line,[:error]
5,2019-06-07,Fri,php undefined salt in on line,[:error]
6,2020-02-29,Sat,user authentication failure for password misma...,[auth_basic:error]
7,2020-03-01,Sun,user authentication failure for password,[auth_basic:error]
8,2020-03-01,Sun,user authentication failure for password,[auth_basic:error]
9,2020-03-01,Sun,user authentication failure for password,[auth_basic:error]
